# IBM Ponder This - February 2020 - Challenge
by Walter Sebastian Gisler

## Problem Statement

The original problem statement can be found here: https://www.research.ibm.com/haifa/ponderthis/challenges/February2020.html

In the famous Snakes and Ladders game, there is a board with 100 squares. You start at square 0 (just outside of the board) and then proceed to advance based on your move, which is dictacted by the throw of a fair dice. If your piece lands on a square that is at the very bottom of a ladder, you climb it. If your turn lands you at the head of a snake, you slide down to the bottom of its tail. And if your turn takes you out of the board (a square > 100), you stay in the square where you are at the time the dice was thrown.

Your challenge this month is to design a game with 10 ladders and snakes altogether that will lead to an expected number of moves (rounded to the 6th decimal place) of 66.978705.

Provide your answer as a list of 10 [source,target] pairs.

As an example, the standard game of 19, with 9 ladders and 10 snakes: 
[1,38],[4,14],[9,31],[21,42],[28,84],[36,44],[51,67],[71,91],[80,100],[16,6],[47,26],[49,11],[56,53],[62,19],[64,60,],[87,24],[93,73],[95,75],[98,78] has an expected 39.225122 moves (again, rounded to the 6th decimal place).

## Different ways of calculating the expected number of moves

### Simulation

Before I start, I want to verify that I am calculating the expected number of moves correctly. I depend on this calculation later on. Let's start by simulating a game:

In [2]:
# We need to be able to simulate a dice, the choice method is perfect for this

from random import choice

# Let us now specify the game that was given as an example

game = [[1,38],[4,14],[9,31],[21,42],[28,84],[36,44],[51,67],[71,91],[80,100],[16,6],[47,26],[49,11],[56,53],[62,19],[64,60,],[87,24],[93,73],[95,75],[98,78]]

# The "game" format is not so convenient, so let us create a dictionary where each key represents a 
# square that is either a head of snake or the bottom of a ladder and each value represents the tail
# of the corresponding snake or the top of the corresponding ladder

paths = {pair[0]: pair[1] for pair in game}

# Now, this is our method that we can use to simulate a single game

def simulate():
    current_tile = 0
    number_attempts = 0
    while current_tile != 100: # If we are on square 100, the game is finished, in this case number_attempts is returned
        dice = choice([1,2,3,4,5,6]) # Throw a dice
        next_tile = current_tile + dice # This is the square we will land on
        number_attempts += 1 # Every throw of a dice increases the number of attempts by 1
        if next_tile in paths: # If we land on a tile that corresponds to a head of a snake or the end of a ladder, we move
            current_tile = paths[next_tile]
        else: # Otherwise, we stay on that square
            current_tile = next_tile
        if current_tile > 100: # If we throw a number that gets us past square 100, we just stay where we are
            current_tile = current_tile - dice
    return number_attempts

Ok, now let's do a single simulation and see what we get:

In [3]:
print(simulate())

34


That's quite close to 39.225, but we still can't be sure the example is correct. Let's do this 100000 times and see what we get now:

In [7]:
number_simulations = 1000000
total_attempts = 0
for i in range(number_simulations):
    total_attempts += simulate()
print('Expected number of attempts: '+str(total_attempts/number_simulations))

Expected number of attempts: 39.242938


Alright, we are off by around 0.02 but I think this is good enough to be sure that the example is correct. However, this way of calculating the expected number of moves is obviously rather slow and not accurate enough, so we need a better method.

### Markov Chains

The snakes and ladders game can obviously be modeled using markov chains. A fantastic tool! I first came across markov chains in an undergraduate operations research class where we were looking at queing theory. Later on, I encountered them again in many different contexts: maroeconomics, revenue management, machine learning (text recognition for example), price forecasting, sports analytics etc. They are everywhere!

A nice article about Markov chains and the snakes and ladders game can be found here: http://www.datagenetics.com/blog/november12011/

In order to work with Markov chains, we first need to define the transition matrix, which is the matrix that gives the probability of passing from one state to another (i.e. from one square to another). each row index in this matrix corresponds to a start position in the game and each column index corresponds to an end position after making one move from a start position. To faciliate things, I am first defining a method that allows me to calculate where I end up, given a start position, the result of dice roll and a given game:

In [8]:
def calculate_end_field(start_field, dice, game):
    paths = {pair[0]: pair[1] for pair in game}
    if start_field + dice in paths:
        return paths[start_field + dice]
    elif start_field + dice > 100:
        return start_field
    else:
        return start_field + dice

Now, let's define a method that calculates the expected number of moves:

In [9]:
import numpy as np

def calculate_expected_num_attempts(game):
    fields = list(range(101))
    
    # initialize transition matrix
    transition_matrix = [[0 for end_field in fields] for start_field in fields]

    # populate values in transition matrix
    for start_field in fields:
        for dice in [1,2,3,4,5,6]:
            transition_matrix[start_field][calculate_end_field(start_field, dice, game)] +=  1/6
            
    # Once the player arrives on the last square, the game is finished, the player is absorbed:
    transition_matrix[100][100] = 0 
    
    # Convert this to a numpy matrix, and remove the last row and column
    tm = np.matrix(transition_matrix)
    tm1 = np.delete(tm, 100, 0)
    tm2 = np.delete(tm1, 100, 1)
    
    # And calculate the expect number of moves! (for explanation take a look at the link above)
    a = (np.identity(100)-tm2)**-1
    e = np.sum(a[0])
    return e

Let's test that with the example that was given in the problem statement:

In [10]:
game = [[1,38],[4,14],[9,31],[21,42],[28,84],[36,44],[51,67],[71,91],[80,100],[16,6],[47,26],[49,11],[56,53],[62,19],[64,60],[87,24],[93,73],[95,75],[98,78]]
e = calculate_expected_num_attempts(game)
print(e)

39.2251223082349


Perfect. That confirms that this method is working correctly. Plus: the calculation seems pretty fast, so we could actually try some kind of local search. Let's try:

## Solution Attempt 1: Local Search

First, we need a method to copy a given game and also a method to make small modifications to a game. For modifications, we will remove one snake or ladder and then reinsert a new rand one.

In [16]:
def copy_game(original):
    return [[el[0], el[1]] for el in original]
    
def modify_game(game):
    new_game = copy_game(game)
    modification = choice([1,2,3]) # randomly chose what kind of modification we want to do
    sl1 = choice([0,1,2,3,4,5,6,7,8,9]) # chose a first snake or ladder
    sl2 = choice(list(set([0,1,2,3,4,5,6,7,8,9])-{sl1})) # chose a second snake or ladder
    if modification == 1:
        # completely redo a ladder
        new_game[sl1][0] = choice(list(range(1, 100)))
        new_game[sl1][1] = choice(list(range(new_game[sl1][0], 101)))
    if modification == 2:
        # completely redo a snake
        new_game[sl1][0] = choice(list(range(2, 100)))
        new_game[sl1][1] = choice(list(range(1, new_game[sl1][0])))
    if modification == 3:
        # completely redo a ladder and a snake
        new_game[sl1][0] = choice(list(range(1, 100)))
        new_game[sl1][1] = choice(list(range(new_game[sl1][0], 101)))
        new_game[sl2][0] = choice(list(range(2, 100)))
        new_game[sl2][1] = choice(list(range(1, new_game[sl2][0])))
    # Make sure that the there are no overlaps (we don't want a square to have more than one start or end of a snake or ladder)
    flat = []
    for element in new_game:
        flat += element
    if len(flat) == len(set(flat)):
        return new_game
    # If there are overlaps, the old game is returned without modifications
    return game

We will use simulated annealing, so we also need a scoring function and an acceptance function. We could easily replace the accept_neighbor function to turn this into a tabu search etc.

In [20]:
def score(solution):
    return -abs(calculate_expected_num_attempts(solution) - 66.978705)

from math import exp
from random import random
def accept_neighbor(current, neighbor, current_score, neighbor_score, temperature):
    enumber = (neighbor_score-current_score)/temperature
    p = exp(enumber)
    rand = random()
    next = current
    if(rand < p):
        next = neighbor
        current_score = neighbor_score
    return next, current_score

Now let's add the main method for simulated annealing:

In [23]:
from time import time

def simulated_annealing(starting_solution, start_temperature, t_min, alpha, beta):
    best_candidate = starting_solution
    total_evaluated = 0
    total_improvements = 0
    best_score = 100000
    current_candidate = best_candidate
    next_candidate = None
    temperature = start_temperature
    current_score = score(current_candidate)
    best_score = score(best_candidate)
    counter = 0
    print("Initial score: %f" % best_score)
    print("Initial temperature: %f" % temperature)
    start_time = time()
    while(temperature > t_min and best_score != 0):
        counter += 1
        next_candidate = modify_game(current_candidate)
        next_score = score(next_candidate)
        current_candidate, current_score = accept_neighbor(current_candidate, next_candidate, current_score, next_score, temperature)
        total_evaluated += 1
        if(current_score > best_score):
            total_improvements += 1
            counter = 0
            best_candidate = current_candidate
            best_score = current_score
        if(counter == beta):
            counter = 0
            temperature = temperature*alpha
            print("New temperature: %f, Improvements: %d, Best score: %f, Current score: %f" % (temperature, total_improvements, best_score, current_score));

    print('Final score: %f' % best_score)
    print("Runtime: %d seconds" % (time()-start_time))
    print("Schedules evaluated per second: %f" % (total_evaluated/(time()-start_time)))
    print("Number of improving moves: %d" % total_improvements)
    print(best_candidate)
    return best_candidate

And give it a run:

In [25]:
mygame = [[4,14],[12,25],[20,35],[28,45],[27,47],[63,56],[73,66],[83,76],[93,86],[98,91]]

solution = simulated_annealing(mygame, 0.1, 0.01, 0.99, 100)

Initial score: -34.200464
Initial temperature: 0.100000
New temperature: 0.099000, Improvements: 7, Best score: -0.067913, Current score: -0.067913
New temperature: 0.098010, Improvements: 7, Best score: -0.067913, Current score: -0.067913
New temperature: 0.097030, Improvements: 7, Best score: -0.067913, Current score: -0.067913
New temperature: 0.096060, Improvements: 7, Best score: -0.067913, Current score: -0.117041
New temperature: 0.095099, Improvements: 8, Best score: -0.066358, Current score: -0.113802
New temperature: 0.094148, Improvements: 8, Best score: -0.066358, Current score: -0.113802
New temperature: 0.093207, Improvements: 10, Best score: -0.055481, Current score: -0.055481
New temperature: 0.092274, Improvements: 10, Best score: -0.055481, Current score: -0.055481
New temperature: 0.091352, Improvements: 11, Best score: -0.045340, Current score: -0.133719
New temperature: 0.090438, Improvements: 11, Best score: -0.045340, Current score: -0.216518
New temperature: 0.0

New temperature: 0.040882, Improvements: 16, Best score: -0.000151, Current score: -0.020358
New temperature: 0.040473, Improvements: 16, Best score: -0.000151, Current score: -0.103703
New temperature: 0.040068, Improvements: 16, Best score: -0.000151, Current score: -0.055219
New temperature: 0.039668, Improvements: 16, Best score: -0.000151, Current score: -0.055219
New temperature: 0.039271, Improvements: 16, Best score: -0.000151, Current score: -0.055055
New temperature: 0.038878, Improvements: 16, Best score: -0.000151, Current score: -0.055055
New temperature: 0.038490, Improvements: 16, Best score: -0.000151, Current score: -0.055055
New temperature: 0.038105, Improvements: 16, Best score: -0.000151, Current score: -0.055055
New temperature: 0.037724, Improvements: 16, Best score: -0.000151, Current score: -0.055055
New temperature: 0.037346, Improvements: 16, Best score: -0.000151, Current score: -0.078169
New temperature: 0.036973, Improvements: 16, Best score: -0.000151, Cu

New temperature: 0.016713, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.016546, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.016381, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.016217, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.016055, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.015894, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.015735, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.015578, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.015422, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.015268, Improvements: 16, Best score: -0.000151, Current score: -0.009534
New temperature: 0.015115, Improvements: 16, Best score: -0.000151, Cu

Let's verify the result:

In [26]:
print(calculate_expected_num_attempts(solution))

66.97885561478508


Pretty good for a "simple" approach. But unfortunately it "doesn't cut the mustard" as my friend B would say in such situations. I made this a bit more sophistaced, chose the neighborhoods more strategically and added reheats. The accuracy quickly improved to about e-5, but that was it. Even after running it for a whole day it did not find a solution that satisfied the criteria. Obviously, there must be a better way of doing this. Alternatively, I could imagine formulating the expected number of attempts as linear equations. This would result in a large system of linear equations that could be solved for the expected number of attempts to get to the last square. Once we have a formula, we could use Gurobi/ Cplex/ Cpoptimizer to determine where the ladders/ snakes should be located. Unfortunately, it is not easy to solve for this variable. Even a system with just 10 squares takes really long to solve using Matlab and the formula is really long. The 100 square system could not be solved in a reasonable amount of time. Unfortunately, I didn't have more time to spend on this, so I finish this challenge unsolved, with a solution that is close to what is required, but not close enough.